In [ ]:
from tensorflow.keras import models, optimizers, losses
from tensorflow.keras.utils import Progbar
from utils import *
from network import *
from loss import *
from matplotlib import pyplot as plt
import cv2

In [ ]:
G = res_unet(n_slice=6, case=2)

print("\n===================================\n")

In [ ]:
G.compile(optimizer=optimizers.Adam(lr=0.0001, epsilon=1e-8), loss=Custom_L2_SSIM)

In [ ]:
train_low, train_high, val_low, val_high, test_low = data_loader_v2('./interpolation/', norm=True)
print("Train X's shape : ", train_low.shape)
print("Train Y's shape : ", train_high.shape)
print("Validation X's shape : ", val_low.shape)
print("Validation Y's shape : ", val_high.shape)
print("Test X's shape : ", test_low.shape)

In [ ]:
shuffle_idx = np.random.choice(len(train_low), len(train_low), replace=False)
train_low = train_low[shuffle_idx]
train_high = train_high[shuffle_idx]
batch = 2
steps = len(train_low)//batch +1

In [ ]:
G.fit(train_low, train_high, batch_size=batch, epochs = 1500)

In [ ]:
idx = 10
test_in = train_low[idx:idx+1]
test= G.predict(test_in)
print(test_in.max())

plot_fn(test, train_high[idx:idx+1])

In [ ]:
train, _ = A.predict(train_low)
train_mse = MSE(train, train_high)

print(train_mse.mean())

In [ ]:
idx = 0
val_in = val_low
val, _ = A.predict(test_in)
#print(test_in.max())
print(val.shape)
print(val_high.shape)
#plot_fn(test, val_high[idx:idx+1])
to_save_val = np.transpose(val, [0, 3, 1, 2]).reshape((156, 320, 256))
to_save_label = np.transpose(val_high, [0, 3, 1, 2]).reshape((156, 320, 256))
val_mse = MSE(val, val_high)
print(val_mse.mean())

In [ ]:
test_in = test_low
test, _ = A.predict(test_in)
#print(test_in.max())
print(test.shape)
#plot_fn(test, val_high[idx:idx+1])
to_save_test = np.transpose(test, [0, 3, 1, 2]).reshape((len(test)*6, 320, 256))
np.save("./result/base_dense/total", to_save_test)

In [ ]:
def plot_fn_single(img):
    mdx = len(img)-1
    def plot(idx=0):
        plt.imshow(img[idx], cmap='gray', vmin=0, vmax=img.max())
        plt.xlabel("%f ~ %f"%(img[idx].min(), img[idx].max()))
    interact(plot, idx=(0, mdx, 1))

In [ ]:
plot_fn_single(to_save_test)

In [ ]:
def view_data(img):
    plt.figure(figsize=(16, 10))
    plt.subplot(131)
    plt.imshow(img[...,0], cmap='gray')
    plt.xlabel("%f ~ %f"%(img[...,0].min(), img[...,0].max()))
    plt.subplot(132)
    plt.imshow(img[...,1], cmap='gray')
    plt.xlabel("%f ~ %f"%(img[...,1].min(), img[...,1].max()))
    plt.subplot(133)
    plt.imshow(img[...,2], cmap='gray')
    plt.xlabel("%f ~ %f"%(img[...,2].min(), img[...,2].max()))
    plt.show()

In [ ]:
def plot_fn(pred, label, mode='val'):
#     if mode != 'test':
        
    plt.figure(figsize=(16, 10))
    for i in range(6):
        plt.subplot(2, 6, i+1)
        plt.xlabel("%f"%(pred[0,...,i].max()))
        plt.imshow(pred[0,...,i], cmap='gray', vmin = 0, vmax = label.max())
        plt.subplot(2, 6, i+1+6)
        plt.title("%f"%(label[0,...,i].max()))
        plt.imshow(label[0,...,i], cmap='gray', vmin = 0, vmax = label.max())
    plt.show()

In [ ]:
def MSE(x, y):
    """
    x, y : (batch, height, width, channel)
    """
    return np.mean(np.square(x-y), axis=(1, 2))

# def PSNR(x, y):
#     """
#     x, y : (batch, height, width, channel)
#     """
#     return 
# def SSIM(x, y):
#     """
#     x, y : (batch, height, width, channel)
#     """
#     I_xy = 2*np.mean(x, axis=(1, 2))*np.mean(y, axis=(1, 2))